In [1]:
#data cleaning
with open('/content/drive/MyDrive/peterpan.txt') as f:
  peterpan_txt=f.read()


In [2]:
#more cleaning
# define punctuation
punctuations = '''!()-[]{};:'\"\,<>./?@#$%^&*_~“”'''


# remove punctuation from the string
no_punct = ""
for char in peterpan_txt:
   if char not in punctuations:
       no_punct = no_punct + char

In [3]:
peterpan_final=no_punct.lower()
strip_peterpan=peterpan_final.strip('"')

In [4]:
len(strip_peterpan)

243004

In [5]:
#overwrite the content of peterpan
text_file = open("/content/drive/MyDrive/peterpan.txt", "w")
n = text_file.write(strip_peterpan)
text_file.close()

In [6]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [9]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()
from pyspark import SparkFiles

In [10]:

paperpan_df = spark.read.text('/content/drive/MyDrive/peterpan.txt')


In [11]:

paperpan_data = spark.sparkContext.textFile('/content/drive/MyDrive/peterpan.txt')
paperpan_data_split = paperpan_data.flatMap(lambda line: line.split(" "))


paperpan_data_split.collect()

['all',
 'children',
 'except',
 'one',
 'grow',
 'up',
 'they',
 'soon',
 'know',
 'that',
 'they',
 'will',
 'grow',
 'up',
 'and',
 'the',
 'way',
 'wendy',
 'knew',
 'was',
 'this',
 'one',
 'day',
 'when',
 'she',
 'was',
 'two',
 'years',
 'old',
 'she',
 'was',
 'playing',
 'in',
 'a',
 'garden',
 'and',
 'she',
 'plucked',
 'another',
 'flower',
 'and',
 'ran',
 'with',
 'it',
 'to',
 'her',
 'mother',
 'i',
 'suppose',
 'she',
 'must',
 'have',
 'looked',
 'rather',
 'delightful',
 'for',
 'mrs',
 'darling',
 'put',
 'her',
 'hand',
 'to',
 'her',
 'heart',
 'and',
 'cried',
 '“oh',
 'why',
 'cant',
 'you',
 'remain',
 'like',
 'this',
 'for',
 'ever”',
 'this',
 'was',
 'all',
 'that',
 'passed',
 'between',
 'them',
 'on',
 'the',
 'subject',
 'but',
 'henceforth',
 'wendy',
 'knew',
 'that',
 'she',
 'must',
 'grow',
 'up',
 'you',
 'always',
 'know',
 'after',
 'you',
 'are',
 'two',
 'two',
 'is',
 'the',
 'beginning',
 'of',
 'the',
 'end',
 'of',
 'course',
 'they',
 'l

In [13]:
paperpan_data_cnt = paperpan_data_split.map(lambda word: (word, 1))

In [14]:

from operator import add
paperpan_data_cnt_all = paperpan_data_cnt.reduceByKey(add)


In [15]:
paperpan_data_cnt_all.saveAsTextFile('peterpan_wordcount.txt')

In [16]:
spark.read.text('peterpan_wordcount.txt').show()

+----------------+
|           value|
+----------------+
|    ('all', 240)|
|('children', 86)|
|  ('except', 19)|
|    ('one', 206)|
|     ('grow', 8)|
|     ('up', 144)|
|   ('they', 569)|
|    ('soon', 30)|
|   ('that', 582)|
|    ('will', 82)|
|   ('and', 1361)|
|   ('the', 2303)|
|  ('wendy', 282)|
|    ('knew', 68)|
|     ('day', 18)|
|    ('she', 574)|
|     ('old', 25)|
| ('playing', 11)|
|      ('a', 921)|
| ('another', 33)|
+----------------+
only showing top 20 rows

